In [6]:
###########################Final analysis of intruder and clutter classification for all the Three outputs 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC 
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
#import nltk as nl
####simple explanation of 3 NN classification
import pandas as pd
import numpy as np
import math as m
from numpy import linalg as LA
from collections import Counter
from scipy.stats.mstats import zscore
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import pylab as pl
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import precision_recall_fscore_support
class Results_Framework(object):
    
    def __init__(self,featureDimension,sheetnameValTrain,sheetnameValTest,class_column):
        self.featureDimension = featureDimension
        self.sheetnameValTrain = sheetnameValTrain
        self.sheetnameValTest = sheetnameValTest
        self.class_column = class_column
    def read_data(self,directoryPathTrain,directoryPathTest):
#        print "The Feature Dimension is:" + str(featureDimension)
        print "The SheetValueTrain is:" + str(sheetnameValTrain)
        print "The SheetValueTestis:" + str(sheetnameValTest)
        val_train = pd.read_excel(directoryPathTrain,sheetname=sheetnameValTrain)
        val_test  = pd.read_excel(directoryPathTest,sheetname=sheetnameValTest)
        ######################################################################################################
        val_data_train = val_train.iloc[:,self.featureDimension]
        val_label_train = val_train.iloc[:,self.class_column]
        val_data_test = val_test.iloc[:,self.featureDimension]
        val_label_test = val_test.iloc[:,self.class_column]
        #####################################################################################################
        X_scaler = StandardScaler()
        train_data = X_scaler.fit_transform(val_data_train)
        test_data = X_scaler.transform(val_data_test)
        print X_scaler.mean_
        print X_scaler.var_
        ###########################################################################################
        #################################################################################
        train_data = np.asarray(train_data)
        
        #print train_labels
        #print len(train_labels)
        #for i in range(len(val_label_train)):
        #    if val_label_train.iloc[i]==0:
        #        val_label_train.iloc[i]=1
        train_labels = list(val_label_train)    
        test_data = np.asarray(test_data)
        
        #for i in range(len(val_label_test)):
        #    if val_label_test.iloc[i]==0:
        #        val_label_test.iloc[i]=1
        test_labels = list(val_label_test)        
        return train_data,train_labels,test_data,test_labels
    
    def Class_validation(self,train_data,train_output,test_data,test_labels_modified):
        Classifier_estimates = [];
        cv = KFold(n=len(train_data),n_folds=5,random_state=12345,shuffle=True)
    
        param_grid_decison = {'max_depth': [None],"max_features":[None],"min_samples_leaf": [1,3,10],"criterion": ["gini","entropy"]}
        param_grid_random = {"max_depth": [3, None],
              "max_features": ["auto"],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]
                 }
        param_test_GB = {'max_depth':range(5,16,2),'n_estimators':range(50,100,10)}
        parameter_grid_svm = [
          {'C': [0.05,0.09,0.03,0.01,1,0.001,0.002,1.0,5.0,10,50.0,100,200,400,800,1600,3200], 'gamma': [0.067,0.06,0.07,0.065,0.01,0.02,0.05,0.08,0.03,0.1,0.001,0.002,0.1],'kernel': ['rbf']}
         
    ]
        param_grid_ada = {
              "n_estimators": range(100,150,10),
             
             }
        param_grid_KNN =  [
            {'n_neighbors': [5,7,9,11]}, {'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}
        ]
        #clf1 = GridSearchCV(estimator= RandomForestClassifier(n_estimators=20,random_state=1), param_grid=param_grid_random)
        #clf3 = GridSearchCV(estimator = GradientBoostingClassifier(max_features='sqrt', subsample=0.8, random_state=1), param_grid = param_test_GB)
        clf1 = GridSearchCV(estimator = tree.DecisionTreeClassifier(random_state=1),param_grid=param_grid_decison)
        clf2 = GridSearchCV(estimator= SVC(random_state=1,class_weight='balanced'), param_grid=parameter_grid_svm)
        #clf4 = GridSearchCV(estimator = AdaBoostClassifier(random_state=1), param_grid=param_grid_ada)
        clf3 = GridSearchCV(estimator = KNN(), param_grid=param_grid_KNN)

        y_train = train_output
        
        models = []
        models.append(('DecisionTree',clf1 ))
        models.append(('SVM', clf2))
        models.append(('KNN', clf3))
        #models.append(('ADA',clf4))
        #models.append(('SVM', clf5))
        #models.append(('KNN', clf6))
    
# evaluate each model in turn
        results = []
        names = []
        scoring = 'f1_micro'
        best_params = []
        for name, model in models:
            kfold = model_selection.KFold(n_splits=5, random_state=12345)
            cv_results = model_selection.cross_val_score(model, train_data, y_train, cv=kfold, scoring=scoring)
            results.append(cv_results)
            names.append(name)
            model.fit(train_data,y_train)
            print "The best parameter estimates for"+str(name)+"is :"+str(model.best_estimator_)
            Classifier_estimates.append(model.best_estimator_)
            msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
            print msg
        return  Classifier_estimates 
    def Classification_process(self,train_data_modified,train_labels_modified,test_data_modified,test_labels_True,Classifier_estimates):
        models = []
        #print "The classifier:"
        #print Classifier_estimates[0]
        clf1 = Classifier_estimates[0]
        clf2 = Classifier_estimates[1]
        clf3 = Classifier_estimates[2]
        
    
        models.append(('DecisionTree',clf1 ))
        models.append(('SVM', clf2))
        models.append(('KNN', clf3))
        
        # evaluate each model in turn
        results_accuracy = []
        names = []
        scoring = 'f1'
        best_params = []
        confusion_matrix_values = []
        results_precision = []
        results_recall = []
        results_f1score = []
        test_labels_predicted_list = []
        for name, model in models:
            model.fit(train_data_modified,train_labels_modified)
            #print len(train_data_modified)
            #print len(train_labels_modified)
            test_labels_predicted = model.predict(test_data_modified)
            #print len(test_labels_predicted)
            #print len(test_labels_predicted)
            target_names = ['Clutter','Human','Animal']
            print "The Iteration is :"+ str(1)+ "The name of the classifier is:"+ str(name) 
            precision,recall,f_score,support = precision_recall_fscore_support(test_labels_True, test_labels_predicted,average='macro')
            results_precision.append(100 * precision)
            results_recall.append(100 * recall)
            results_f1score.append(100 * f_score)
            print (classification_report(test_labels_True, test_labels_predicted, target_names=target_names))
            count = 0
            for i in range(len(test_labels_predicted)):
                test_labels_predicted_list.append(test_labels_predicted[i])
                if(test_labels_predicted[i] == test_labels_True[i]):
                    count = count + 1
                    
                
            print "The accuracy is:" + str(float(float(count)/float(len(test_labels_predicted)))) 
            accuracy = 100 * (float(float(count)/float(len(test_labels_predicted))));
            cm = confusion_matrix(test_labels_True, test_labels_predicted, labels =None)
            print "The confusion_matrix is:" 
            print(cm) 
            confusion_matrix_values.append((np.reshape(cm, (1, 9))).tolist()[0])
            results_accuracy.append(accuracy)
        return results_accuracy,confusion_matrix_values,results_precision,results_recall,results_f1score
        
    def main_method(self,directoryTrain,directoryTest):
        train_data_modified,train_labels_modified,test_data_modified,test_labels_modified = results_framework.read_data(directoryTrain,directoryTest);
        Classifier_estimates = results_framework.Class_validation(train_data_modified,train_labels_modified,test_data_modified,test_labels_modified)
        results_accuracy,confusion_matrix_values,results_precision,results_recall,results_f1score = results_framework.Classification_process(train_data_modified,train_labels_modified,test_data_modified,test_labels_modified,Classifier_estimates)
        return results_accuracy,confusion_matrix_values,results_precision,results_recall,results_f1score          



directory_train = "C:\\Users\\smart\\Dropbox\\Abhimanyu\\Algorithm_Comparison\\Matlab_Code\\Matlab_All_Codes\\Camera_Analysis\\"
#directory_train = "C:\\Users\\WSNLab\\Dropbox_Tarun\\Dropbox\\Abhimanyu\\Algorithm_Comparison\\Matlab_Code\\Matlab_All_Codes\\Camera_Analysis"
file_name_train = "Feature_final_case_for_all_human_and_animal_with_mog_kfold.xlsx"
directory_test = directory_train
file_name_test = file_name_train
df = pd.read_excel(directory_train+file_name_train)
columns = len(df.columns)
featureDimension = range(18,26)
class_column = columns-2
accuracy_all = []
precision_all = []
recall_all = []
f1score_all = []
confusion_matrix_sum_all = [] 
count_even = 0;
count_odd = 1;
for i in range(5):    
    sheetnameValTrain = count_even;
    sheetnameValTest  = count_odd;
    print "##############################################################################################"
    print "The Analysis of Camera Three class accuracy" + str(i)
    print "##############################################################################################"
    results_framework = Results_Framework(featureDimension,sheetnameValTrain,sheetnameValTest,class_column)
    #accuracy,confusion_matrix_sum = 
    results_accuracy,confusion_matrix_values,results_precision,results_recall,results_f1score = results_framework.main_method(os.path.join(directory_train,file_name_train),os.path.join(directory_test,file_name_test))
    accuracy_all.append(results_accuracy)
    precision_all.append(results_precision)
    recall_all.append(results_recall)
    f1score_all.append(results_f1score)
    
    df_accuracy=pd.DataFrame(accuracy_all,columns=['DecisionTree_acc','SVM_acc','KNN_acc'])
    df_precision=pd.DataFrame(precision_all,columns=['DecisionTree_acc','SVM_acc','KNN_acc'])
    df_recall=pd.DataFrame(recall_all,columns=['DecisionTree_acc','SVM_acc','KNN_acc'])
    df_f1score=pd.DataFrame(f1score_all,columns=['DecisionTree_acc','SVM_acc','KNN_acc'])
    
    confusion_matrix_sum_all.append(confusion_matrix_values)
    df_confusion_matrix_sum_all=pd.DataFrame(confusion_matrix_sum_all,columns=['DecisionTree_acc','SVM_acc','KNN_acc'])
    
    print "##############################################################"
    print "The final accuracies of classifier are:"
    print df_accuracy
    print "##############################################################"
    print "The final confusion matrix of classifier are:"
    print df_confusion_matrix_sum_all
    ###########printing of the whole confusion matrix##################################
    if(count_even==8):
        for i in range(np.shape(df_confusion_matrix_sum_all)[1]):
            xlist = []
            xlist = list((df_confusion_matrix_sum_all.iloc[:,i]))
            a = [sum(x) for x in zip(*xlist)]
            print "name of the classifier is:" + str(df_confusion_matrix_sum_all.columns[i])
            print "The total confusion matrix is:"
            print np.reshape(a,(3,3))
            #print "The Sucessful Target Detection is for the classifier for intruder:" + str(df_confusion_matrix_sum_all.columns[i]) + str((a[0])/(a[0] + a[1]))
            #print "The Sucessful False Alarms Raisec for the classifier are:" + str(df_confusion_matrix_sum_all.columns[i]) + str((a[2])/(a[2] + a[3]))
            #print "The missed scenarios are:" + str(df_confusion_matrix_sum_all.columns[i]) + str((a[1])/(a[0] + a[1]))
            #print "The mean accuracy is:" + df_accuracy.iloc[:,1].astype('float64').mean()
            #print "The variance of the accuracy is:" + df_accuracy.iloc[:,1].astype('float64').var()
    else:
        count_even = count_even + 2
        count_odd = count_odd + 2
    
    
    

##############################################################################################
The Analysis of Camera Three class accuracy0
##############################################################################################
The SheetValueTrain is:0
The SheetValueTestis:1
[  6415.70403587  10048.58295964  18365.83408072  12111.76681614
   3820.7309417    5823.0896861   13662.10313901  14169.62331839]
[  6.32906263e+07   2.25469232e+08   5.19052825e+08   4.01204753e+08
   3.92332996e+07   1.19657957e+08   3.59623258e+08   4.92420063e+08]
The best parameter estimates forDecisionTreeis :DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=3,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')
DecisionTree: 0.932828 (0.031290)
The best parameter estimates forSVMis :SVC(C=200, cac

In [2]:
print "The mean and the variance for " + str(df_confusion_matrix_sum_all.columns[0])
print df_accuracy.iloc[:,0].mean()
print df_accuracy.iloc[:,0].var()
print "The mean and the variance for " + str(df_confusion_matrix_sum_all.columns[1])
print df_accuracy.iloc[:,1].mean()
print df_accuracy.iloc[:,1].var()
print "The mean and the variance for " + str(df_confusion_matrix_sum_all.columns[2])
print df_accuracy.iloc[:,2].mean()
print df_accuracy.iloc[:,2].var()


The mean and the variance for DecisionTree_acc
93.9534062429
7.04646339279
The mean and the variance for SVM_acc
93.6085668717
22.2157259709
The mean and the variance for KNN_acc
94.6294144452
3.365646557


In [3]:
print "The mean and the variance for precision " + str(df_confusion_matrix_sum_all.columns[0])
print df_precision.iloc[:,0].mean()
print df_precision.iloc[:,0].var()
print "The mean and the variance for precision" + str(df_confusion_matrix_sum_all.columns[1])
print df_precision.iloc[:,1].mean()
print df_precision.iloc[:,1].var()
print "The mean and the variance for precision" + str(df_confusion_matrix_sum_all.columns[2])
print df_precision.iloc[:,2].mean()
print df_precision.iloc[:,2].var()

The mean and the variance for precision DecisionTree_acc
93.4566865883
5.37831235644
The mean and the variance for precisionSVM_acc
93.6887224975
12.2532902901
The mean and the variance for precisionKNN_acc
94.7464843586
7.2003937851


In [4]:
print "The mean and the variance for recall" + str(df_confusion_matrix_sum_all.columns[0])
print df_recall.iloc[:,0].mean()
print df_recall.iloc[:,0].var()
print "The mean and the variance for recall" + str(df_confusion_matrix_sum_all.columns[1])
print df_recall.iloc[:,1].mean()
print df_recall.iloc[:,1].var()
print "The mean and the variance for recall" + str(df_confusion_matrix_sum_all.columns[2])
print df_recall.iloc[:,2].mean()
print df_recall.iloc[:,2].var()
print "The mean and the variance for recall" + str(df_confusion_matrix_sum_all.columns[3])


The mean and the variance for recallDecisionTree_acc
93.3821733822
12.7013908922
The mean and the variance for recallSVM_acc
92.909035409
32.6367120237
The mean and the variance for recallKNN_acc
93.634004884
4.75391127577


IndexError: index 3 is out of bounds for axis 0 with size 3

In [5]:
###################Calculation_of_fscore
print "The mean and the variance for fscore" + str(df_confusion_matrix_sum_all.columns[0])
print df_f1score.iloc[:,0].mean()
print df_f1score.iloc[:,0].var()
print "The mean and the variance for fscore" + str(df_confusion_matrix_sum_all.columns[1])
print df_f1score.iloc[:,1].mean()
print df_f1score.iloc[:,1].var()
print "The mean and the variance for fscore" + str(df_confusion_matrix_sum_all.columns[2])
print df_f1score.iloc[:,2].mean()
print df_f1score.iloc[:,2].var()
print "The mean and the variance for fscore" + str(df_confusion_matrix_sum_all.columns[3])

The mean and the variance for fscoreDecisionTree_acc
93.0612443307
9.4553675374
The mean and the variance for fscoreSVM_acc
93.0261818071
24.9297023234
The mean and the variance for fscoreKNN_acc
93.9101785873
6.66965084777


IndexError: index 3 is out of bounds for axis 0 with size 3